In [1]:
import findspark
import sentry_sdk

sentry_sdk.init(
    dsn="https://examplePublicKey@o0.ingest.sentry.io/0",

    # Set traces_sample_rate to 1.0 to capture 100%
    # of transactions for performance monitoring.
    # We recommend adjusting this value in production.
    traces_sample_rate=1.0
)
findspark.init()
findspark.find()

'c:\\Users\\greeshma.ganta\\Anaconda3\\lib\\site-packages\\pyspark'

In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("mail_notify").getOrCreate()

In [3]:
!pip install great_expectations===0.14.12

  Using cached great_expectations-0.14.12-py3-none-any.whl (4.9 MB)


  Attempting uninstall: great-expectations
    Found existing installation: great-expectations 0.15.31
    Uninstalling great-expectations-0.15.31:
      Successfully uninstalled great-expectations-0.15.31


In [6]:
import sys
import os
#sys.path.insert(1,"C:\\Users\\abhinav.anand\\Desktop\\dq_check")
from great_expectations.data_context.types.base import DatasourceConfig
from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.data_context.types.base import DataContextConfig, DatasourceConfig, FilesystemStoreBackendDefaults
from great_expectations.data_context import BaseDataContext
from great_expectations.exceptions import DataContextError
from great_expectations.core.expectation_configuration import ExpectationConfiguration
from great_expectations.render.renderer import ValidationResultsPageRenderer
from ruamel import yaml
from multiprocessing.pool import Pool, ThreadPool
import multiprocessing
from multiprocessing.pool import ThreadPool
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import concat_ws
from pyspark.sql.window import Window
import json
import re
import time
from datetime import datetime as dt
import datetime
import configparser
import logging

In [14]:
#%run "C:/Users/abhinav.anand/Desktop/dq_check/custom_expectations/expect_column_values_to_contain_valid_email.py"

In [15]:
#%run "C:/Users/abhinav.anand/Desktop/dq_check/custom_expectations/expect_column_values_to_not_contain_special_characters.py"

In [10]:
CONFIG= configparser.ConfigParser()
CONFIG.read("C:\Users\greeshma.ganta\Desktop\dqc/config/config.ini")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (Temp/ipykernel_12448/400148552.py, line 2)

In [17]:
print(CONFIG["DQ_CHECK_CONFIG"]["RULES_REPO"])

C:/Users/abhinav.anand/Desktop/rules.json


In [18]:
file_date = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d-%H-%M-%S')
log_file='C:/Users/abhinav.anand/Desktop/dq_check/logs/'+'dq_ge_'+str(file_date)+'.log'
format_log = logging.Formatter('%(asctime)s  %(levelname)-8s [%(name)s:%(lineno)d]  %(message)s')

logger = logging.getLogger('dq_ge')
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler(log_file, mode='a')

sh = logging.StreamHandler()
sh.setLevel(logging.DEBUG)

fh.setFormatter(format_log)
sh.setFormatter(format_log)
if (logger.hasHandlers()):
  logger.handlers.clear()
logger.addHandler(sh)
logger.addHandler(fh)

In [19]:
class GeDQFrameWork:
  
    #Create_data_source_config
    def _data_source_config(self,connector, batch_id, engine):
      data_source_config = DatasourceConfig(
          class_name="Datasource",
          execution_engine={"class_name": engine },
          data_connectors={
            connector : {
              "module_name": "great_expectations.datasource.data_connector",
              "class_name": "RuntimeDataConnector",
              "batch_identifiers": [str(batch_id)] }
          }
        )
      logger.info(f'{connector} - Datasource Genereted')
      return data_source_config

    
    #Create_project_config
    def _create_project_config(self, spark_datasource_name, spark_data_source_config,pandas_datasource_name, pandas_data_source_config):
        project_config = DataContextConfig( datasources={
            spark_datasource_name : spark_data_source_config, pandas_datasource_name:pandas_data_source_config}, 
                                           stores={ "expectations_store": {
                  "class_name": "ExpectationsStore",
                  "store_backend": { "class_name": "TupleFilesystemStoreBackend",
                  "base_directory": CONFIG["DQ_CHECK_CONFIG"]["EXPECTATION_PATH"]}, },
                  "validations_store": {
                      "class_name": "ValidationsStore",
                      "store_backend": { "class_name": "TupleFilesystemStoreBackend",
                      "base_directory": CONFIG["DQ_CHECK_CONFIG"]["VALIDATION_PATH"],
                        },},
                   "evaluation_parameter_store": { "class_name": "EvaluationParameterStore"},
                                                    
              }, 
        validation_operators={ "action_list_operator": {
                "class_name": "ActionListValidationOperator",
                "action_list": [{
                        "name": "store_validation_result",
                        "action": {"class_name": "StoreValidationResultAction"},
                    },
                   {
                         "name": "store_evaluation_params",
                         "action": {"class_name": "StoreEvaluationParametersAction"},
                     },
                     {
                        "name": "update_data_docs",
                        "action": {"class_name": "UpdateDataDocsAction"},
                    }
#                     {
#                       "name": "send_email_on_validation_result",
#                       "action": {
#                         "class_name": "EmailAction",
#                         "notify_on": "all",
#                         "use_tls": True,
#                         "use_ssl": False,
#                         "renderer": {
#                           "module_name": "great_expectations.render.renderer.email_renderer",
#                           "class_name": "EmailRenderer"
#                         },
#                         "smtp_address": "smtp-mail.outlook.com",
#                         "smtp_port": 587,
#                         "sender_login": "abhinavv.anand@outlook.com",
#                         "sender_password": "****",
#                         "sender_alias": "abhinavv.anand@outlook.com",
#                         "receiver_emails": "abhinavanandpnbe@gmail.com",
#                       }
#                     }
                     
                ]} },
           data_docs_sites = {
                    "local_site": {
                    "class_name": "SiteBuilder",
                    "store_backend": { "class_name": "TupleFilesystemStoreBackend",
                                     "base_directory": CONFIG["DQ_CHECK_CONFIG"]["DATA_DOCS_PATH"]},
                    "site_index_builder": {"class_name": "DefaultSiteIndexBuilder",
                                     "show_cta_footer": True, },} },
                                           
             store_backend_defaults=FilesystemStoreBackendDefaults(root_directory=CONFIG["DQ_CHECK_CONFIG"]["VALIDATION_PATH"])
            )
        logger.info('Project Configuration Created')
        return project_config

    #create_batch_request
    def _create_batch_request(self, datasource_name, data_connector_name, data_asset_name,data_batch,batch_id, run_id):
      batch_request = RuntimeBatchRequest(
        datasource_name= datasource_name,
        data_connector_name= data_connector_name,
        data_asset_name= data_asset_name,
        runtime_parameters= { "batch_data": data_batch },
        batch_identifiers= {batch_id: run_id})
      logger.info(f'{datasource_name} - Batchrequest Created !!!')
      return batch_request

    #add_expectations_to_the_suite
    def _add_expectations_to_suite(self,data,suite):
      data_meta = None
      for row in data.collect():
        if row['source_type'] == 'csv': 
          data_meta = {"file_name": row["source"]}
        kwargs = json.loads('{'+str(row["parameter"])+'}')
        expectation_configuration = ExpectationConfiguration(expectation_type = row["validation_name"], kwargs =kwargs, meta = data_meta)
        suite.add_expectation(expectation_configuration=expectation_configuration,match_type="domain")
        logger.info(f'Expectation - {row["validation_name"]} - Created for:  {row["source"]}')
      return 
    
    #create_validadtor
    def _create_validator(self, batch_request,context, suite):
      data_validator = context.get_validator(batch_request=batch_request,expectation_suite=suite)
      logger.info("Validator Created")
      data_validator.save_expectation_suite(discard_failed_expectations=False)
      return data_validator

    #read_dataframe
    def _read_data_frame(self, path):
      data_frame = spark.read.format("csv").options(inferSchema = "true" ,multiline = "true",header = "true", sep= ",",quote="\"",escape="\"").load(path)
      return data_frame
#     def _read_data_json(self, path):
#     # Opening JSON file
#       f = open(path)
#       dataf = json.load(f)
#       return dataf
    
    
    def _get_failed_expectations(self,resultss,df):
      failed_expectations = list()
      if isinstance(resultss, list):
          failed_expectations = list(__builtins__.filter(lambda x: x['success']== False, resultss))
      return self._bad_records_count(failed_expectations,df)
        

    def _bad_records_count(self,failed_expectations,df):
        bad_records_df = df.filter("1 == 2")##check
        good_records_df_count = None
        for f in failed_expectations:
            column_name = f['expectation_config']['kwargs'].get('column')
            unexpected_list = f['result'].get('unexpected_list')

            if f['expectation_config'].get('expectation_type') == 'expect_column_values_to_not_be_null':
                data_df = df.filter(col(column_name).isNull())


            if column_name is not None:
                if unexpected_list is not None and len(unexpected_list) > 0:   
                    temp_df = df.filter(col(column_name).isin(set(unexpected_list)))
                    if f['expectation_config'].get('expectation_type') == 'expect_column_values_to_not_be_null':
                        temp_df = data_df
                        
                    #data_df.show() 
                    bad_records_df = bad_records_df.union(temp_df)
                    good_records_df = df.subtract(bad_records_df)
                    try:
                        if good_records_df.count() > 0:
#                            good_records_df_count = good_records_df.dropDuplicates().count()
                            good_records_df_count = good_records_df.dropDuplicates().count()
                            #logger.info(f'Good Records Count: {good_records_df_count}')
                    except Exception as e:
                        logger.error(f'{e} Exception occured in Getting good records')
#         bad_records = bad_records_df.coalesce(1).write.option("header", "true").mode("append").csv("C:/Users/abhinav.anand/Desktop/dq_check/"+"bad_rec"+'_'+str(file_date)+".csv")
#         good_records = good_records_df.coalesce(1).write.option("header", "true").mode("append").csv("C:/Users/abhinav.anand/Desktop/dq_check/"+"good_rec"+'_'+str(file_date)+".csv")            
        logger.info(f'Good Records Count: {good_records_df_count}')         
        if bad_records_df.count()   > 0:
            bad_records_df_count = bad_records_df.count()
#            bad_records_df_count = bad_records_df.dropDuplicates().count()
            
            return bad_records_df_count
    
          
    def _get_bad_records(self,validation_ids, validations_results_dict, data_df):
              results_dict = None
              bad_rec_count = None
              for ind,val in enumerate(validation_ids): 
                logger.info(f'Validation Id:{val}')
                if ind == 0:
                    results_dict = validations_results_dict['run_results'][str(val)]['validation_result']['results']
                if ind >= 1:
                    for i in validations_results_dict['run_results'][str(val)]['validation_result']['results']:
                        results_dict.append(i)
              try:
                if results_dict != None:
                  bad_rec_count = self._get_failed_expectations(resultss=results_dict, df=data_df)
                  logger.info(f'Bad Records Count: {bad_rec_count}')
              except Exception as e:
                logger.error(f'{e} Exception occured in Getting bad records')
              return bad_rec_count

    
    #Mandatory_optional_field_checking
    #Mandatory_optional_field_checking
    def _mandatory_option_check(self, joined_table):
        df_list=[]
        combinedDF = joined_table.select( col("id"),col("parameter"), col("parameter_name"),col("parameter_required"))
        df1 = combinedDF.withColumn('new', concat_ws(',',split(regexp_replace(regexp_replace(col('parameter'),' ',''),'"',"'"), ':')))
        quoted_page_name = concat(lit("'"), col("parameter_name"), lit("'"))
        res = df1.withColumn("IDmatch", when(col("new").contains(quoted_page_name), True).otherwise(False))
        res.withColumn("size",when(size(split(regexp_replace(col("parameter"),':([^,]*)', "~#~$1"),'~#~'))==1,1).otherwise(size(split(regexp_replace(col("parameter"),':([^,]*)', "~#~$1"),'~#~'))-1)).registerTempTable('res')
        if (res.filter((res.IDmatch == "False") & (res.parameter_required == 'MANDATORY')).count() >0 ):
            df_list.append(res.select(col("id"),lit('MANDATORY').alias('error')).filter((res.IDmatch == "False") & (res.parameter_required == 'MANDATORY') ))
        if spark.sql("select id,IDmatch,size from res group by id,IDmatch,size having count(*) != size and IDmatch == 'true'").count() > 0:
            df_list.append(spark.sql("select id ,'OPTIONAL' from res group by id,IDmatch,size having count(*) != size and IDmatch == 'true'"))
        return df_list
    
    def _check_column(self,source_table,combined_table):
      list_1 = []
      combined_table = combined_table.withColumn("table_columns",lit(','.join(source_table.columns)))
      if combined_table.filter(col("parameter").contains('"column"')).count() >0 :
          combined_table_filterd_column = combined_table.filter(col("parameter").contains('"column"'))
          combined_table_filterd_column = combined_table_filterd_column.withColumn('new', regexp_replace(regexp_replace(split(split('parameter',',')[0],':')[1],'"',''),' ',''))
          res = combined_table_filterd_column.withColumn("column_match", when(col("table_columns").contains(col("new")), True).otherwise(False))      
          if res.filter(col("column_match") == False).count() >0: list_1.append(res.filter(col("column_match") == False))
      if combined_table.filter(col("parameter").contains('"column_list"')).count() >0 :
          combined_table_filterd_column_list = combined_table.filter(col("parameter").contains('"column_list"'))
          combined_table_filterd_column_list = combined_table_filterd_column_list.withColumn('new',explode(split(regexp_replace(regexp_replace(regexp_replace(regexp_replace(split(col('parameter'),':')[1],'"',""),'[',''),']',''),' ',''),',')))
          res_column_list = combined_table_filterd_column_list.withColumn("column_match", when(col("table_columns").contains(col("new")), True).otherwise(False))
          if res_column_list.filter(col("column_match") == False).count() >0: list_1.append(res_column_list.filter(col("column_match") == False))
      if combined_table.filter(col("parameter").contains('"column_A"')).count() >0 :
          combined_table_filterd_column_mul = combined_table.filter(col("parameter").contains('"column_A"') | col("parameter").contains('"column_B"'))
          combined_table_filterd_column_mul = combined_table_filterd_column_mul.withColumn('new',explode(split(regexp_replace(regexp_replace(concat(split(split('parameter',',')[0],':')[1],lit("~#~"),split(split('parameter',',')[1],":")[1]),'"',''),' ',''),'~#~')))
          res_column_mul = combined_table_filterd_column_mul.withColumn("column_match", when(col("table_columns").contains(col("new")), True).otherwise(False))
          if res_column_mul.filter(col("column_match") == False).count() >0: list_1.append(res_column_mul.filter(col("column_match") == False))
      return list_1
    
    
    
    #read_rules_repository
    def _get_rules_repo(self, file_name):
      logger.info("Read Rules Repository Files - Started")
      dq_validation = self._read_data_frame(path=CONFIG["DQ_CHECK_CONFIG"]["DQ_VALIDATION"])
      source_validation = self._read_data_frame(path=CONFIG["DQ_CHECK_CONFIG"]["SOURCE_VALIDATION"])
      source_meta = self._read_data_frame(path=CONFIG["DQ_CHECK_CONFIG"]["SOURCE_META"])
      joined_table = self._read_data_frame(path=CONFIG["DQ_CHECK_CONFIG"]["RULES_REPO"])
      validation_parameter = self._read_data_frame(path=CONFIG["DQ_CHECK_CONFIG"]["VALIDATION_LIBRARY_PARAMETER"])
      joined_table = source_validation.alias("sv").join(dq_validation.alias('dv'), source_validation.dqvalidation_id == dq_validation.id,"inner").join(
          source_meta.alias('sm'),source_validation.source_meta_id == source_meta.id,"left")
      mandatory_optional_check = source_validation.alias("sv").join(
         validation_parameter.alias('vp'),validation_parameter.dqvalidation_id == source_validation.dqvalidation_id,"left")
      man_df = self._mandatory_option_check(mandatory_optional_check)
      if len(man_df) == 0: 
          logger.info('Mandatory Optional Check Completed')
      else: 
        logger.info('Issue in Mandatory Optional Check...')
        raise Exception("Parameter missing or wrong entry in the rule repo")
      add_db_table = joined_table.withColumn("source", concat_ws("",col('file_name')))
      db_tables = add_db_table.filter(add_db_table.source == str(f'{file_name}'))

      return db_tables
    

    def _pre_process(self,df,validation_data):
      config_data = {}
      list_return=[]
      for (each_key, each_val) in CONFIG.items("PRE_PROCESS_CONFIG"):
        config_data[each_key] = each_val
        pre_process_list = validation_data.select('parameter','validation_name').collect()
        for pre_process_item in pre_process_list:
          if pre_process_item['validation_name'] in config_data.keys() :
            column_name = str(pre_process_item['parameter'].split(',')[0].split(':')[1]).replace('"','').replace(' ','')
            if df.schema[column_name].dataType != 'DoubleType' and config_data[pre_process_item['validation_name']]=='DoubleType':
              df=df.withColumn(column_name, df[column_name].cast(DoubleType()))
            if df.schema[column_name].dataType != 'IntegerType' and config_data[pre_process_item['validation_name']]=='IntegerType':
              df=df.withColumn(column_name, df[column_name].cast(IntegerType()))
      list_return.append(df)
      if validation_data.filter(col("validation_name")=='expect_column_values_to_change_between').count()>0:
        data_list=validation_data.filter(col("validation_name")=='expect_column_values_to_change_between').collect()
        expectaion=data_list[0]
        column_name = (((expectaion['parameter'].split(',')[0]).split(':')[1]).replace('"','')).replace(' ','')
        from_val = (((expectaion['parameter'].split(',')[1]).split(':')[1]).replace('"','')).replace(' ','')
        to_val = (((expectaion['parameter'].split(',')[2]).split(':')[1]).replace('"','')).replace(' ','')
        my_window = Window.partitionBy().orderBy(column_name)
        df = df.withColumn(column_name+"_diff", lead(col(column_name)).over(my_window))
        df=df.withColumn(column_name+"_diff",col(column_name+"_diff")-col(column_name))
        param = '"column" : "'+column_name+'_diff" , "min_value" : '+str(from_val)+' , "max_value" : '+str(to_val)
        validation_data=validation_data.withColumn('parameter', regexp_replace(col('parameter'),str(expectaion['parameter']), param))
        validation_data=validation_data.withColumn('validation_name', regexp_replace(col('validation_name'),str(expectaion['validation_name']), 'expect_column_values_to_be_between'))
        validation_data=validation_data.withColumn('type',when(col("validation_name") == lit('expect_column_values_to_be_between'),regexp_replace(col("type"),'pandas','spark')).otherwise(col("type")))
        list_return[0]=df
        list_return.append(validation_data)
      return list_return
    
    def actual_process(self,file_name):
      logger.info("Framework has been Started")
      logger.info("actual process has been triggered")
        
      """
      step 1: Read rules repository
      step 2: initialize dataContext with Project configuration
      step 3: Batch creation
      step 4: Create expectation Suite
      step 5: Add expectations to the Expectation suite
      step 6: Creation of Validator
      step 7: Validate data
      """
      
      rules_repo = self._get_rules_repo(file_name)
      logger.info(f"Rules Repository Dataframe Generated")
      if rules_repo.rdd.isEmpty() == False:
          logger.info("Rules Repository Dataframe Have the data")
          spark_datasource_config = self._data_source_config(connector= "SparkDataconnector",
                                                             batch_id = "spark_source",engine = "SparkDFExecutionEngine")
          pandas_datasource_config = self._data_source_config(connector= "PandasDataconnector",
                                                              batch_id = "pandas_source",engine = "PandasExecutionEngine") 

          project_config=self._create_project_config(spark_datasource_name="spark_source", 
                                                     spark_data_source_config=spark_datasource_config,
                                                     pandas_datasource_name= "pandas_source", pandas_data_source_config = pandas_datasource_config)
          context = BaseDataContext(project_config=project_config)
          logger.info("Context Created with Project Configuration")
          rules_source = rules_repo.dropDuplicates(['source']).collect()
          run_id = f'{datetime.datetime.now()}'
          try:
            if rules_source[0]['source_type'] == 'table':
              logger.info("Source type - Table")
              data_df = spark.table(rules_source[0]['source'])
              check_column = self._check_column(data_df,rules_repo)
              if len(check_column) ==0: logger.info("Column Check Completed")                         
              else: raise Exception("Column not present in the Source Table")
              logger.info("PreProcessing Started")
              data_list = self._pre_process(data_df,rules_repo)
              if len(data_list)==2 : 
                data_df = data_list[0]
                rules_repo = data_list[1]
              elif len(data_list)==1 :
                data_df = data_list[0]
              logger.info("PreProcessing Completed")
            elif rules_source[0]['source_type'] == 'csv':
              data_df = spark.read.format(rules_source[0]['source_type']).options(inferSchema = "true",header = "true", sep= ",",quote="\"",
                                                                                  escape="\"").load(rules_source[0]['source_path'])
          except Exception as e:
            logger.error(f"{e} - Exception Occured in Reading the Source data")
            raise Exception("Exit")
          data_type = [i['type'] for i in rules_repo.select(col('type')).collect()]
          spark_data = rules_repo.filter(col('type') == 'spark')
          pandas_data = rules_repo.filter(col('type') == 'pandas')
          validator = []
          try:
            if 'spark' in data_type: 
              batch_request = self._create_batch_request(datasource_name="spark_source",
                                                         data_connector_name="SparkDataconnector", 
                                                         data_asset_name=rules_source[0]['source'],
                                                         data_batch=data_df,batch_id="spark_source", 
                                                         run_id= run_id)
              suite = context.create_expectation_suite(rules_source[0]['source'], overwrite_existing=True)
              logger.info(f'{rules_source[0]["source"]} - Spark Expectation Suite Created !!!')
              self._add_expectations_to_suite(data=spark_data,suite=suite)
              validator.append(self._create_validator(batch_request=batch_request,context= context, suite=suite))
            if 'pandas' in data_type: 
              pandas_batch_request = self._create_batch_request(datasource_name="pandas_source",
                                                           data_connector_name="PandasDataconnector", 
                                                           data_asset_name=f'{rules_source[0]["source"]}'+'_pandas',
                                                           data_batch=data_df.toPandas(),batch_id="pandas_source", 
                                                           run_id= run_id) 
              pandas_suite = context.create_expectation_suite(f'{rules_source[0]["source"]}'+'_pandas', overwrite_existing=True)
              self._add_expectations_to_suite(data=pandas_data,suite=pandas_suite)
              validator.append(self._create_validator(batch_request=pandas_batch_request,context= context, suite=pandas_suite))
          except Exception as e:
            logger.error(str(e)+"Exception Occured in the Batch Request Creation or Suite Creation!!!")
          results = context.run_validation_operator("action_list_operator", assets_to_validate=validator,result_format = {'result_format' : "COMPLETE",'include_unexpected_rows': True})     
          logger.info('results are being Generated...')
          logger.info(f"{results}")

          if results: 
            eval_rows = data_df.count()
            validation_ids = [res for res in results['run_results']]
            validations_results_dict = results.to_json_dict()
            if validation_ids != []:
              bad_rec_count_ret  =self._get_bad_records(validation_ids= validation_ids, validations_results_dict=validations_results_dict, data_df=data_df)
              if bad_rec_count_ret== None: bad_rec_count_ret==0
              result_format = validations_results_dict['validation_operator_config']['kwargs']['result_format']['result_format']
              results_object = []
              resultss = validations_results_dict['run_results'][str(validation_ids[0])]['validation_result']
              val_time = resultss['meta']['validation_time']
              match = re.search(r'\d{4}\d{2}\d{2}', val_time)
              val_date = dt.strptime(match.group(), '%Y%m%d').date()
              validation_time = val_date.strftime("%Y-%m-%d")
              file_name = resultss['results'][0]['expectation_config']['meta']['file_name']
              success = resultss['success']
              for f in resultss['results']:
                if f['expectation_config']['expectation_type'] == "expect_column_value_to_be_in_normal_range":
                    print("inn")
                    f.update({'validation_time' : validation_time})
                    results_object.append(f)
                else:
                    f.update({'result' : {'element_count' : f['result'].get('element_count'),'overall_success':success,'result_format':result_format, 'unexpected_count' : f['result'].get('unexpected_count')},'validation_time' : validation_time})
                    results_object.append(f)
                    
                
              resultss['statistics'].update({'success':success, 'validation_time' : validation_time})
              resultss['statistics'].update({'unexpected_rows': bad_rec_count_ret,'evaluated_rows':eval_rows,'file_name':file_name})
              stat = {'statistics' : resultss['statistics'] }  
              
              results_object.append(stat)
              if len(validation_ids) > 1:
                stats = validations_results_dict['run_results'][str(validation_ids[1])]['validation_result']
                stats['statistics'].update({'success':stats['success'], 'validation_time' : validation_time,'file_name':file_name})
                stat_pandas = {'statistics' : stats['statistics']}  
                results_object.append(stat_pandas)
              result_path = CONFIG["DQ_CHECK_CONFIG"]["RESULT_PATH"]
              file = f'{str(result_path)}/result_{str(file_date)}.json'
              os.makedirs(os.path.dirname(file), exist_ok=True)
              with open(file, 'w') as out:
                for objects in results_object:
#                    out.write("\n")
                    out.write(json.dumps(objects))
                    out.write("\n")
#                     logger.info(json.dumps(objects))
#                    print(objects)
              logger.info(f"File {file} Created in the Directory")
          context.build_data_docs() 
          logger.info("DONE!!!")               
      else:
          logger.error("Rules Repository Don't Have the data for specified file name")

In [20]:
dq_obj = GeDQFrameWork()
try:
    dq_obj.actual_process("movies")
    logger.info("Process Completed Successfully!!!")
except Exception as e:
    logger.error(str(e)+"Error Occured in the actual process")

2022-11-22 11:17:37,781  INFO     [dq_ge:280]  Framework has been Started
2022-11-22 11:17:37,783  INFO     [dq_ge:281]  actual process has been triggered
2022-11-22 11:17:37,786  INFO     [dq_ge:226]  Read Rules Repository Files - Started
2022-11-22 11:17:40,820  INFO     [dq_ge:238]  Mandatory Optional Check Completed
2022-11-22 11:17:40,858  INFO     [dq_ge:294]  Rules Repository Dataframe Generated
2022-11-22 11:17:41,779  INFO     [dq_ge:296]  Rules Repository Dataframe Have the data
2022-11-22 11:17:41,779  INFO     [dq_ge:15]  SparkDataconnector - Datasource Genereted
2022-11-22 11:17:41,782  INFO     [dq_ge:15]  PandasDataconnector - Datasource Genereted
2022-11-22 11:17:41,782  INFO     [dq_ge:80]  Project Configuration Created
2022-11-22 11:17:41,981  INFO     [dq_ge:306]  Context Created with Project Configuration
2022-11-22 11:17:43,525  INFO     [dq_ge:91]  spark_source - Batchrequest Created !!!
2022-11-22 11:17:43,543  INFO     [dq_ge:342]  movies - Spark Expectation Sui

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

2022-11-22 11:17:50,442  INFO     [dq_ge:357]  results are being Generated...
2022-11-22 11:17:50,444  INFO     [dq_ge:358]  {
  "validation_operator_config": {
    "class_name": "ActionListValidationOperator",
    "module_name": "great_expectations.validation_operators",
    "name": "action_list_operator",
    "kwargs": {
      "action_list": [
        {
          "name": "store_validation_result",
          "action": {
            "class_name": "StoreValidationResultAction"
          }
        },
        {
          "name": "store_evaluation_params",
          "action": {
            "class_name": "StoreEvaluationParametersAction"
          }
        },
        {
          "name": "update_data_docs",
          "action": {
            "class_name": "UpdateDataDocsAction"
          }
        }
      ],
      "result_format": {
        "result_format": "SUMMARY",
        "partial_unexpected_count": 20
      }
    }
  },
  "evaluation_parameters": null,
  "run_id": {
    "run_time": "202